<style  type="text/css">
cool {
  width: 100px;
  height: 100px;
  background-color: red;
  position: relative;
  -webkit-animation-name: example; /* Safari 4.0 - 8.0 */
  -webkit-animation-duration: 4s; /* Safari 4.0 - 8.0 */
  -webkit-animation-iteration-count: infinite; /* Safari 4.0 - 8.0 */
  animation-name: example;
  animation-duration: 4s;
  animation-iteration-count: infinite;
}

/* Safari 4.0 - 8.0 */
@-webkit-keyframes example {
  0%   {background-color:red; left:0px; top:0px;}
  25%  {background-color:yellow; left:200px; top:0px;}
  50%  {background-color:blue; left:200px; top:200px;}
  75%  {background-color:green; left:0px; top:200px;}
  100% {background-color:red; left:0px; top:0px;}
}

/* Standard syntax */
@keyframes example {
  0%   {background-color:red; left:0px; top:0px;}
  25%  {background-color:yellow; left:200px; top:0px;}
  50%  {background-color:blue; left:200px; top:200px;}
  75%  {background-color:green; left:0px; top:200px;}
  100% {background-color:red; left:0px; top:0px;}
}
</style>

<b><center>
<span style="font-size: 24pt; line-height: 1.2">
COMS W4111: Introduction to Databases<br>
Fall 2023, Sections 002
</span>
</center></b>
<br>
<p>
<i><center>
<span style="font-size: 20pt; line-height: 1.2">
Homework 2
</span>
</center></i>

# Introduction

This notebook contains HW2. **Both Programming and Nonprogramming tracks should complete this part.** To ensure everything runs as expected, work on this notebook in Jupyter.

Submission instructions:

* You will submit **PDF and ZIP files** for this assignment. Gradescope will have two separate assignments for these.
* For the PDF:
    * The most reliable way to save as PDF is to go to your browser's menu bar and click `File -> Print`. Switch the orientation to landscape mode, and hit save.
    * **MAKE SURE ALL YOUR WORK (CODE AND SCREENSHOTS) IS VISIBLE ON THE PDF. YOU WILL NOT GET CREDIT IF ANYTHING IS CUT OFF.** Reach out for troubleshooting.
* For the ZIP:
    * Zip a folder containing this notebook and any screenshots.
* Further submission instructions may be posted on Edstem.

# Setup

In [6]:
!pip install -q sql
!pip install -q pandas
!pip install -q sqlalchemy
!pip install -q ipython-sql
!pip install -q pymysql

In [1]:
import csv
import copy

import pandas as pd
import pymysql

In [2]:
sql_conn = pymysql.connect(
    user="root",
    password="dbuserdbuser",
    host="localhost",
    port=3306,
    cursorclass=pymysql.cursors.DictCursor,
    autocommit=True)

cur = sql_conn.cursor()
res = cur.execute("SELECT 1")
res = cur.fetchall()
res

[{'1': 1}]

In [22]:

res = cur.execute("DELETE FROM test_db.people WHERE Username = 'test_man';")
res = cur.execute("INSERT INTO test_db.people (Username, Password, Time_Created) VALUES ('test_man', 'test_pwd_2', CURRENT_TIMESTAMP );")


In [ ]:
%%sql

DROP SCHEMA IF EXISTS f23_hw2;
CREATE SCHEMA f23_hw2;
USE f23_hw2;

In [ ]:
%%sql

# The following assumes that you have created everything properly

DELETE FROM f23_hw2.student;
DELETE FROM f23_hw2.employee;

FileNotFoundError: [Errno 2] No such file or directory: 'people_info.csv'

In [3]:
# If you have defined your procedures, functions, tables, and constraints correctly, you can use the function below
# to load your data.

def add_person(p):
    """
    p is a dictionary containing the column values for either a student or an employee.
    """
    
    cur = sql_conn.cursor()
    
    # This function changes the data, converting '' to None.
    # So, we make a copy and change the copy.
    p_dict = copy.copy(p)
    for k,v in p_dict.items():
        if v == '':
            p_dict[k] = None
    
    # Is the person a student?
    #
    if p_dict['employee_type'] is None:
        
        # This provides a hint for what your stored procedure will look like.
        res = cur.callproc("f23_hw2.create_student",
                           # The following are in parameters
                          (p_dict['first_name'],
                          p_dict['middle_name'],
                          p_dict['last_name'],
                           p_dict['email'],
                          p_dict['enrollment_year'],
                           # The following are out parameters for uni.
                          None))
        
        # After the procedure executes, the following query will select the out values.
        res = cur.execute("""SELECT @_f23_hw2.create_student_5""")
        result = cur.fetchall()
    
    # The following does the same for employee.
    elif p_dict['enrollment_year'] is None:
        res = cur.callproc("f23_hw2.create_employee",
                          (p_dict['first_name'],
                          p_dict['middle_name'],
                          p_dict['last_name'],
                           p_dict['email'],
                          p_dict['employee_type'],
                          None))
        res = cur.execute("""SELECT @_f23_hw2.create_student_5""")
        result = cur.fetchall()
    else:
        print("Something went wrong")
        result = None
    
    sql_conn.commit()
    cur.close()
    return result

In [ ]:
for p in people_info:
    add_person(p)

In [ ]:
# Test that we loaded people
%sql SELECT * FROM people

In [ ]:
# Test that we loaded students
%sql SELECT * FROM student

In [ ]:
# Test that we loaded employees
%sql SELECT * FROM employee

## Testing

Add your tests below. You should test

1. Successful execution of each procedure.
2. Execution of update procedures showing that your constraints prevent incorrect data entry and enforce the defined semantics and behavior.
3. ```general_user``` can query the data and call the procedures, but cannot perform ```INSERT```, ```UPDATE```, or ```DELETE```.